In [12]:
import pandas as pd

In [13]:
filename = '/Users/chengxiaowei/Desktop/Python/eas503/Xiaowei_finalProject/xiaowei_data/FlightDelays.csv'
df = pd.read_csv(filename)

In [14]:
df.head()

,schedtime,carrier,deptime,dest,distance,date,flightnumber,origin,weather,dayweek,daymonth,tailnu,delay
0,1455,OH,1455,JFK,184,1/1/2004,5935,BWI,0,4,1,N940CA,ontime
1,1640,DH,1640,JFK,213,1/1/2004,6155,DCA,0,4,1,N405FJ,ontime
2,1245,DH,1245,LGA,229,1/1/2004,7208,IAD,0,4,1,N695BR,ontime
3,1715,DH,1709,LGA,229,1/1/2004,7215,IAD,0,4,1,N662BR,ontime
4,1039,DH,1035,LGA,229,1/1/2004,7792,IAD,0,4,1,N698BR,ontime


In [15]:

df['delay'].value_counts()

delay
ontime     1773
delayed     428
Name: count, dtype: int64

In [16]:
print(df['origin'].unique())
print(df['carrier'].unique())
print(df['dest'].unique())
print(df['weather'].unique())
print(df['dayweek'].unique())
print(df['daymonth'].unique())

print(df['flightnumber'].nunique())
print(df['tailnu'].nunique())

['BWI' 'DCA' 'IAD']
['OH' 'DH' 'DL' 'MQ' 'UA' 'US' 'RU' 'CO']
['JFK' 'LGA' 'EWR']
[0 1]
[4 5 6 7 1 2 3]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
103
549


In [17]:
df.shape

(2201, 13)

In [18]:
df.dtypes

schedtime        int64
carrier         object
deptime          int64
dest            object
distance         int64
date            object
flightnumber     int64
origin          object
weather          int64
dayweek          int64
daymonth         int64
tailnu          object
delay           object
dtype: object

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   schedtime     2201 non-null   int64 
 1   carrier       2201 non-null   object
 2   deptime       2201 non-null   int64 
 3   dest          2201 non-null   object
 4   distance      2201 non-null   int64 
 5   date          2201 non-null   object
 6   flightnumber  2201 non-null   int64 
 7   origin        2201 non-null   object
 8   weather       2201 non-null   int64 
 9   dayweek       2201 non-null   int64 
 10  daymonth      2201 non-null   int64 
 11  tailnu        2201 non-null   object
 12  delay         2201 non-null   object
dtypes: int64(7), object(6)
memory usage: 223.7+ KB


In [20]:
df.isnull().any()

schedtime       False
carrier         False
deptime         False
dest            False
distance        False
date            False
flightnumber    False
origin          False
weather         False
dayweek         False
daymonth        False
tailnu          False
delay           False
dtype: bool

In [21]:
from pathlib import Path
import os
import sqlite3
import pandas as pd

def load_flight_data(csv_path: str):
    print("[1] Loading FlightDelays CSV into DataFrame...")
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} rows, {df.shape[1]} columns.")
    return df

def build_3nf_sqlite_flightdelays(csv_path: str, db_path: str = "flightdelays.db"):
    print("=== BUILDING 3NF SQLITE DATA MODEL (FlightDelays) ===")

    # -------------------------
    # STEP 1: Load CSV
    # -------------------------
    print("\n[STEP 1] Loading CSV into DataFrame...")
    df = load_flight_data(csv_path)

    # Basic cleanup: strip spaces in string columns (safe)
    for c in ["carrier", "origin", "dest", "tailnu", "delay"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

    # -------------------------
    # STEP 2: Create surrogate key flight_id
    # -------------------------
    print("\n[STEP 2] Creating surrogate key flight_id...")
    df = df.reset_index().rename(columns={"index": "flight_id"})
    print("flight_id added.")

    # -------------------------
    # STEP 3: Build dimension tables
    # -------------------------
    print("\n[STEP 3] Building dimension tables...")

    # (A) carrier dimension
    carrier_dim = df[["carrier"]].drop_duplicates().reset_index(drop=True)
    carrier_dim["carrier_id"] = carrier_dim.index + 1
    print(f"dim_carrier: {len(carrier_dim)} unique carriers")

    # (B) airport dimension: union of origin + dest
    airports = pd.concat([df["origin"], df["dest"]], ignore_index=True).dropna()
    airport_dim = pd.DataFrame({"code": airports.unique()})
    airport_dim = airport_dim.sort_values("code").reset_index(drop=True)
    airport_dim["airport_id"] = airport_dim.index + 1
    print(f"dim_airport: {len(airport_dim)} unique airports")

    # (C) tail dimension (optional but we include it)
    tail_dim = df[["tailnu"]].drop_duplicates().reset_index(drop=True)
    tail_dim["tail_id"] = tail_dim.index + 1
    print(f"dim_tail: {len(tail_dim)} unique tail numbers")

    # -------------------------
    # STEP 4: Merge IDs back into main df
    # -------------------------
    print("\n[STEP 4] Merging dimension IDs into main DataFrame...")
    df = df.merge(carrier_dim, on="carrier", how="left")
    df = df.merge(tail_dim, on="tailnu", how="left")

    df = df.merge(
        airport_dim.rename(columns={"code": "origin", "airport_id": "origin_airport_id"}),
        on="origin",
        how="left",
    )
    df = df.merge(
        airport_dim.rename(columns={"code": "dest", "airport_id": "dest_airport_id"}),
        on="dest",
        how="left",
    )

    # -------------------------
    # STEP 5: Create 3NF DataFrames
    # -------------------------
    print("\n[STEP 5] Creating 3NF DataFrames (dim tables + fact table)...")

    df_dim_carrier = carrier_dim.rename(columns={"carrier": "code"})[["carrier_id", "code"]]
    df_dim_airport = airport_dim[["airport_id", "code"]]
    df_dim_tail = tail_dim[["tail_id", "tailnu"]]

    # Fact table: keep original numeric columns + foreign keys + label
    # We store delay as TEXT now; later you can convert to 0/1 during modeling.
    df_flight = df[[
        "flight_id",
        "date",
        "schedtime",
        "deptime",
        "distance",
        "flightnumber",
        "weather",
        "dayweek",
        "daymonth",
        "carrier_id",
        "origin_airport_id",
        "dest_airport_id",
        "tail_id",
        "delay",
    ]].copy()

    print("3NF DataFrames created.")

    # -------------------------
    # STEP 6: Create SQLite DB + schema
    # -------------------------
    print("\n[STEP 6] Creating SQLite database and tables...")

    if os.path.exists(db_path):
        print("Existing DB found. Removing...")
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    print("Running SQL schema creation script...")
    cur.executescript(
        """
        DROP TABLE IF EXISTS flight;
        DROP TABLE IF EXISTS dim_tail;
        DROP TABLE IF EXISTS dim_airport;
        DROP TABLE IF EXISTS dim_carrier;

        CREATE TABLE dim_carrier (
            carrier_id INTEGER PRIMARY KEY,
            code TEXT NOT NULL UNIQUE
        );

        CREATE TABLE dim_airport (
            airport_id INTEGER PRIMARY KEY,
            code TEXT NOT NULL UNIQUE
        );

        CREATE TABLE dim_tail (
            tail_id INTEGER PRIMARY KEY,
            tailnu TEXT NOT NULL UNIQUE
        );

        CREATE TABLE flight (
            flight_id INTEGER PRIMARY KEY,

            date TEXT,
            schedtime INTEGER,
            deptime INTEGER,
            distance INTEGER,
            flightnumber INTEGER,
            weather INTEGER,
            dayweek INTEGER,
            daymonth INTEGER,

            carrier_id INTEGER NOT NULL,
            origin_airport_id INTEGER NOT NULL,
            dest_airport_id INTEGER NOT NULL,
            tail_id INTEGER NOT NULL,

            delay TEXT NOT NULL,

            FOREIGN KEY (carrier_id) REFERENCES dim_carrier(carrier_id),
            FOREIGN KEY (origin_airport_id) REFERENCES dim_airport(airport_id),
            FOREIGN KEY (dest_airport_id) REFERENCES dim_airport(airport_id),
            FOREIGN KEY (tail_id) REFERENCES dim_tail(tail_id)
        );
        """
    )
    print("Tables created.")

    # -------------------------
    # STEP 7: Insert data
    # -------------------------
    print("\n[STEP 7] Inserting data into SQLite database...")

    print("Inserting dim_carrier...")
    cur.executemany(
        "INSERT INTO dim_carrier (carrier_id, code) VALUES (?, ?)",
        list(df_dim_carrier.itertuples(index=False, name=None)),
    )

    print("Inserting dim_airport...")
    cur.executemany(
        "INSERT INTO dim_airport (airport_id, code) VALUES (?, ?)",
        list(df_dim_airport.itertuples(index=False, name=None)),
    )

    print("Inserting dim_tail...")
    cur.executemany(
        "INSERT INTO dim_tail (tail_id, tailnu) VALUES (?, ?)",
        list(df_dim_tail.itertuples(index=False, name=None)),
    )

    print("Inserting flight fact table...")
    cur.executemany(
        """
        INSERT INTO flight (
            flight_id, date, schedtime, deptime, distance, flightnumber,
            weather, dayweek, daymonth,
            carrier_id, origin_airport_id, dest_airport_id, tail_id,
            delay
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        list(df_flight.itertuples(index=False, name=None)),
    )

    conn.commit()
    conn.close()

    print("\n=== DONE! SQLite DB created at:", db_path, "===\n")


# ---- RUN ----
csv_path = "/Users/chengxiaowei/Desktop/Python/eas503/Xiaowei_finalProject/xiaowei_data/FlightDelays.csv"  # <-- 改成你的路径
db_path = "/Users/chengxiaowei/Desktop/Python/eas503/Xiaowei_finalProject/xiaowei_data/flightdelays.db"


build_3nf_sqlite_flightdelays(csv_path, db_path=db_path)


=== BUILDING 3NF SQLITE DATA MODEL (FlightDelays) ===

[STEP 1] Loading CSV into DataFrame...
[1] Loading FlightDelays CSV into DataFrame...
Loaded 2201 rows, 13 columns.

[STEP 2] Creating surrogate key flight_id...
flight_id added.

[STEP 3] Building dimension tables...
dim_carrier: 8 unique carriers
dim_airport: 6 unique airports
dim_tail: 549 unique tail numbers

[STEP 4] Merging dimension IDs into main DataFrame...

[STEP 5] Creating 3NF DataFrames (dim tables + fact table)...
3NF DataFrames created.

[STEP 6] Creating SQLite database and tables...
Existing DB found. Removing...
Running SQL schema creation script...
Tables created.

[STEP 7] Inserting data into SQLite database...
Inserting dim_carrier...
Inserting dim_airport...
Inserting dim_tail...
Inserting flight fact table...

=== DONE! SQLite DB created at: /Users/chengxiaowei/Desktop/Python/eas503/Xiaowei_finalProject/xiaowei_data/flightdelays.db ===



In [22]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(db_path)

print("Tables:", pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))

print("\nCounts:")
for t in ["dim_carrier","dim_airport","dim_tail","flight"]:
    n = pd.read_sql(f"SELECT COUNT(*) AS n FROM {t};", conn)
    print(t, int(n["n"][0]))

print("\nSample JOIN (flight + carrier + airports):")
q = """
SELECT
  f.flight_id,
  f.date,
  f.schedtime,
  c.code AS carrier,
  a1.code AS origin,
  a2.code AS dest,
  f.distance,
  f.weather,
  f.dayweek,
  f.daymonth,
  f.delay
FROM flight f
JOIN dim_carrier c ON f.carrier_id = c.carrier_id
JOIN dim_airport a1 ON f.origin_airport_id = a1.airport_id
JOIN dim_airport a2 ON f.dest_airport_id = a2.airport_id
LIMIT 5;
"""
print(pd.read_sql(q, conn))
conn.close()


Tables:           name
0  dim_carrier
1  dim_airport
2     dim_tail
3       flight

Counts:
dim_carrier 8
dim_airport 6
dim_tail 549
flight 2201

Sample JOIN (flight + carrier + airports):
   flight_id      date  schedtime carrier origin dest  distance  weather  \
0          0  1/1/2004       1455      OH    BWI  JFK       184        0   
1          1  1/1/2004       1640      DH    DCA  JFK       213        0   
2          2  1/1/2004       1245      DH    IAD  LGA       229        0   
3          3  1/1/2004       1715      DH    IAD  LGA       229        0   
4          4  1/1/2004       1039      DH    IAD  LGA       229        0   

   dayweek  daymonth   delay  
0        4         1  ontime  
1        4         1  ontime  
2        4         1  ontime  
3        4         1  ontime  
4        4         1  ontime  
